In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.impute import SimpleImputer

# Charger les données
train_data = pd.read_csv('train_Insurance.csv')
test_data = pd.read_csv('test_Insurance.csv')

# Analyse exploratoire des données
print("Nombre de lignes :", len(train_data))
print("Nombre de colonnes :", len(train_data.columns))
print("Types de colonnes :\n", train_data.dtypes)
print("Valeurs manquantes :\n", train_data.isna().sum())




# Exploration initiale
L = list(train_data.columns)
L_num = list(train_data.select_dtypes(exclude='object').columns)
L_d = list(train_data.select_dtypes(include='object').columns)
print("Liste des colonnes :")
print(L)
print("Liste des colonnes numériques :")
print(L_num)
print("Liste des attributs discrets :")
print(L_d)

# Statistiques descriptives
print("Statistiques descriptives :\n", train_data.describe(include="all"))
descriptives_numeric = train_data.describe()
descriptives_all = train_data.describe(include='all')
print("Statistiques descriptives (attributs numériques) :")
print(descriptives_numeric)
print("Statistiques descriptives (tous les attributs) :")
print(descriptives_all)

# Identification des valeurs manquantes
missing_values = train_data.isnull().sum()
total_missing = train_data.isnull().sum().sum()
print("Valeurs manquantes par attribut :")
print(missing_values)
print("Nombre total de valeurs manquantes :")
print(total_missing)









Nombre de lignes : 5012
Nombre de colonnes : 13
Types de colonnes :
 Customer Id            object
YearOfObservation       int64
Insured_Period        float64
Residential             int64
Building_Painted       object
Building_Fenced        object
Garden                 object
Settlement             object
Building Dimension    float64
Building_Type          object
NumberOfWindows        object
Geo_Code               object
Claim                  object
dtype: object
Valeurs manquantes :
 Customer Id            0
YearOfObservation      0
Insured_Period         0
Residential            0
Building_Painted       0
Building_Fenced        0
Garden                 4
Settlement             0
Building Dimension    77
Building_Type          0
NumberOfWindows        0
Geo_Code              73
Claim                  0
dtype: int64
Liste des colonnes :
['Customer Id', 'YearOfObservation', 'Insured_Period', 'Residential', 'Building_Painted', 'Building_Fenced', 'Garden', 'Settlement', 'Building Dim

In [5]:
# Create a copy of the original dataframe
df_copy = train_data.copy()

for col in cols:
    # Calcul des quartiles et de l'écart interquartile
    Q1, Q3 = np.percentile(df_copy[col], [25, 75])
    IQR = Q3 - Q1

    # Calcul des limites pour les valeurs aberrantes
    upper_limit = Q3 + 1.5 * IQR
    lower_limit = Q1 - 1.5 * IQR

    # Identification des outliers
    outliers = df_copy[(df_copy[col] < lower_limit) | (df_copy[col] > upper_limit)]
    
    print(f"Nombre d'outliers pour {col} : {len(outliers)}")

    # Remplacement des valeurs aberrantes par les limites
    df_copy[col] = np.where(df_copy[col] >= upper_limit, upper_limit,
                       np.where(df_copy[col] <= lower_limit, lower_limit, df_copy[col]))


NameError: name 'cols' is not defined

In [ ]:
import numpy as np
import seaborn as sns

# Sélection des colonnes numériques
numeric_columns = ['YearOfObservation', 'Insured_Period', 'Residential', 'Building Dimension']

# Création de boxplots pour chaque colonne numérique
for column in numeric_columns:
    plt.figure(figsize=(8, 6))
    sns.boxplot(x=train_data[column])
    plt.title(f'Boxplot de {column}')
    plt.show()

    # Calcul des limites pour les valeurs aberrantes
    Q1, Q3 = np.percentile(train_data[column], [25, 75])
    IQR = Q3 - Q1
    upper_limit = Q3 + 1.5 * IQR
    lower_limit = Q1 - 1.5 * IQR

    # Identification des valeurs aberrantes
    outliers = train_data[column][(train_data[column] > upper_limit) | (train_data[column] < lower_limit)]
    print(outliers)
    # Affichage du nombre d'outliers
    print(f"Nombre d'outliers pour {column} : {len(outliers)}")
   


    # Traitement des valeurs aberrantes
    train_data[column] = np.where(train_data[column] >= upper_limit, upper_limit,
                          np.where(train_data[column] <= lower_limit, lower_limit, train_data[column]))
    
    

In [ ]:
# Justifications:
# Pour "Garden", la perte d'informations en supprimant les lignes est négligeable.
# Pour "Building Dimension", utiliser la médiane est approprié pour éviter l'influence des valeurs extrêmes.
# Pour "Geo_Code", remplacer par le mode est une option raisonnable étant donné que c'est une variable catégorielle avec relativement peu de modalités manquantes.



# Suppression des instances ayant des valeurs manquantes dans la colonne 'Garden'
#la perte d'information en supprimant des lignes
train_data = train_data.dropna(subset=['Garden'])

# Remplissage des valeurs manquantes dans 'Building Dimension' par la médiane
# La médiane est moins sensible aux valeurs aberrantes que la moyenne.
# Si votre colonne 'Building Dimension' contient des valeurs extrêmes,
# l'utilisation de la médiane pour remplir les valeurs manquantes peut donner de meilleurs résultats.

median_imputer = SimpleImputer(strategy='median')
train_data['Building Dimension'] = median_imputer.fit_transform(train_data['Building Dimension'].values.reshape(-1,1))

# Cette stratégie est souvent utilisée pour les variables catégorielles.
# L’idée est de remplir les valeurs manquantes par la valeur la plus fréquente.
# Cela peut être une bonne stratégie lorsque la variable a une catégorie dominante.
# Remplissage des valeurs manquantes dans 'Geo_Code' par l’élément le plus fréquent
freq_imputer = SimpleImputer(strategy='most_frequent')
train_data['Geo_Code'] = freq_imputer.fit_transform(train_data['Geo_Code'].values.reshape(-1,1))



#  test de nouveau  des valeurs manquantes
missing_values = train_data.isnull().sum()
total_missing = train_data.isnull().sum().sum()
print("Valeurs manquantes par attribut :")
print(missing_values)
print("Nombre total de valeurs manquantes :")
print(total_missing)


# Suppression des instances ayant des valeurs manquantes dans la colonne 'Garden'
test_data = test_data.dropna(subset=['Garden'])

# Remplissage des valeurs manquantes dans 'Building Dimension' par la médiane
# Notez que nous utilisons 'transform' au lieu de 'fit_transform' pour utiliser la médiane calculée sur les données d'entraînement
test_data['Building Dimension'] = median_imputer.transform(test_data['Building Dimension'].values.reshape(-1,1))

# Remplissage des valeurs manquantes dans 'Geo_Code' par l’élément le plus fréquent
# Encore une fois, nous utilisons 'transform' pour utiliser le mode calculé sur les données d'entraînement
test_data['Geo_Code'] = freq_imputer.transform(test_data['Geo_Code'].values.reshape(-1,1))

# Identification des valeurs manquantes
missing_values = test_data.isnull().sum()
total_missing = test_data.isnull().sum().sum()
print("Valeurs manquantes par attribut :")
print(missing_values)
print("Nombre total de valeurs manquantes :")
print(total_missing)



In [ ]:
import numpy as np
import seaborn as sns

# Sélection des colonnes numériques
numeric_columns = ['YearOfObservation', 'Insured_Period', 'Residential', 'Building Dimension']

# Création de boxplots pour chaque colonne numérique
for column in numeric_columns:
    plt.figure(figsize=(8, 6))
    sns.boxplot(x=train_data[column])
    plt.title(f'Boxplot de {column}')
    plt.show()

    # Calcul des limites pour les valeurs aberrantes
    Q1, Q3 = np.percentile(train_data[column], [25, 75])
    IQR = Q3 - Q1
    upper_limit = Q3 + 1.5 * IQR
    lower_limit = Q1 - 1.5 * IQR

    # Identification des valeurs aberrantes
    outliers = train_data[column][(train_data[column] > upper_limit) | (train_data[column] < lower_limit)]
    # Affichage du nombre d'outliers
    print(f"Nombre d'outliers pour {column} : {len(outliers)}")
   


    # Traitement des valeurs aberrantes
    train_data[column] = np.where(train_data[column] >= upper_limit, upper_limit,
                          np.where(train_data[column] <= lower_limit, lower_limit, train_data[column]))


import numpy as np
import seaborn as sns

# Sélection des colonnes numériques
numeric_columns = ['YearOfObservation', 'Insured_Period', 'Residential', 'Building Dimension']

# Création de boxplots pour chaque colonne numérique
for column in numeric_columns:
    plt.figure(figsize=(8, 6))
    sns.boxplot(x=test_data[column])
    plt.title(f'Boxplot de {column}')
    plt.show()

    # Calcul des limites pour les valeurs aberrantes
    Q1, Q3 = np.percentile(test_data[column], [25, 75])
    IQR = Q3 - Q1
    upper_limit = Q3 + 1.5 * IQR
    lower_limit = Q1 - 1.5 * IQR

    # Identification des valeurs aberrantes
    outliers = test_data[column][(test_data[column] > upper_limit) | (test_data[column] < lower_limit)]

    
    # Affichage du nombre d'outliers
    print(f"Nombre d'outliers pour {column} : {len(outliers)}")
    
    # Traitement des valeurs aberrantes
    test_data[column] = np.where(test_data[column] >= upper_limit, upper_limit,
                          np.where(test_data[column] <= lower_limit, lower_limit, test_data[column]))



In [ ]:
print(train_data[train_data.duplicated()])

print(test_data[test_data.duplicated()])


In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import gaussian_kde
import seaborn as sns
import matplotlib.pyplot as plt



# Liste des colonnes pour lesquelles vous voulez créer des KDE
cols = ['YearOfObservation', 'Insured_Period', 'Building Dimension']

for col in cols:
    # Calcul de la moyenne
    moyenne = train_data[col].mean()
    print("Moyenne de " + col + " :", moyenne)

    # Calcul de la médiane
    median = train_data[col].median()
    print("Médiane de " + col + " :", median)

    # Calcul du mode
    train_data.sort_values(by=col, inplace=True)
    x_vals = train_data[col].values
    kde = gaussian_kde(x_vals)
    y_vals = kde(x_vals)
    valeur_mode = x_vals[np.argmax(y_vals)]
    print("Mode de " + col + " :", valeur_mode)

   # Ajouter des lignes verticales pour la moyenne, la médiane et le mode
    plt.axvline(moyenne, color='r', linestyle='--', label='Moyenne')
    plt.axvline(median, color='g', linestyle='-', label='Médiane')
    plt.axvline(valeur_mode, color='b', linestyle='-', label='Mode')

    # Créer un histogramme pour la colonne
    sns.histplot(x=col, data=train_data, bins=3, kde=True, stat='density')
    plt.title("Histogramme avec KDE de " + col)
    plt.xlabel(col)
    plt.ylabel("Densité de probabilité")
    plt.show()






In [ ]:
    #le coefficient d’asymétrie vous aide à comprendre la distribution de vos données et peut vous donner des indications sur la manière
#de prétraiter vos données pour l’analyse.


# YearOfObservation : Le coefficient d’asymétrie est de 0.35, ce qui indique une légère asymétrie à droite.
# Cela signifie que la queue de la distribution est légèrement étirée vers les valeurs plus grandes que la médiane.

# Insured_Period : Le coefficient d’asymétrie est de -1.09, ce qui indique une asymétrie à gauche.
# Cela signifie que la queue de la distribution est étirée vers les valeurs plus petites que la médiane.

# Building Dimension : Le coefficient d’asymétrie est de 1.13, ce qui indique une asymétrie à droite.
# Cela signifie que la queue de la distribution est étirée vers les valeurs plus grandes que la médiane.



# Liste des colonnes pour lesquelles vous voulez calculer le coefficient d'asymétrie
cols = ['YearOfObservation', 'Insured_Period', 'Building Dimension']

for col in cols:
    # Calcul du coefficient d'asymétrie
    skewness = train_data[col].skew()
    print("Coefficient d'asymétrie pour " + col + " :", skewness)

    # Interprétation du coefficient d'asymétrie
    if skewness > 0:
        print("La distribution de " + col + " est asymétrique à droite.")
    elif skewness < 0:
        print("La distribution de " + col + " est asymétrique à gauche.")
    else:
        print("La distribution de " + col + " est symétrique.")


# Liste des colonnes pour lesquelles vous voulez calculer le coefficient d'asymétrie
cols = ['YearOfObservation', 'Insured_Period', 'Building Dimension']

for col in cols:
    # Calcul du coefficient d'asymétrie
    skewness = test_data[col].skew()
    print("Coefficient d'asymétrie pour " + col + " :", skewness)

    # Interprétation du coefficient d'asymétrie
    if skewness > 0:
        print("La distribution de " + col + " est asymétrique à droite.")
    elif skewness < 0:
        print("La distribution de " + col + " est asymétrique à gauche.")
    else:
        print("La distribution de " + col + " est symétrique.")


In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt



# Liste des colonnes pour lesquelles vous voulez créer des histogrammes
cols = ['YearOfObservation', 'Residential', 'Building_Painted', 'Building_Fenced', 'Garden', 'Settlement', 'Building_Type', 'NumberOfWindows', 'Geo_Code', 'Claim']

for col in cols:
    plt.figure(figsize=(10, 6))
    sns.countplot(x=col, data=train_data)
    plt.title('Histogramme de ' + col)
    plt.xlabel(col)
    plt.ylabel('Fréquence')
    plt.show()


In [ ]:
sns.countplot(x='Claim', data=train_data, hue ='YearOfObservation')

In [ ]:
sns.countplot(x='Claim', data=train_data, hue ='Settlement')

In [ ]:
import seaborn as sns

sns.countplot(x='Building_Painted', hue='Claim', data=train_data)



In [ ]:
sns.countplot(x='Building_Fenced', hue='Claim', data=train_data)


In [ ]:
sns.countplot(x='Garden', hue='Claim', data=train_data)


In [ ]:
sns.countplot(x='Settlement', hue='Claim', data=train_data)

In [ ]:
from sklearn.preprocessing import LabelEncoder

# Les modèles de machine learning ont généralement besoin de travailler avec des données numériques,
# donc les variables catégorielles doivent être transformées en nombres avant de pouvoir être utilisées pour l’entraînement d’un modèle.

# Création d'une instance de LabelEncoder
label_encoder = LabelEncoder()

# Application de l'encodage de label à la colonne 'Claim'
train_data['Claim'] = label_encoder.fit_transform(train_data['Claim'])


# Application de l'encodage de label à la colonne 'Claim'
test_data['Claim'] = label_encoder.fit_transform(test_data['Claim'])


In [ ]:

from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler
import pandas as pd
#Nombre d'outliers pour Building Dimension : 396

# Cette méthode est efficace pour minimiser l'impact des valeurs aberrantes,
# ce qui pourrait être le cas ici étant donné l'asymétrie.


# Pour la standardisation RobustScaler
robust_scaler = RobustScaler()
train_data['Building Dimension'] = robust_scaler.fit_transform(train_data[['Building Dimension']])

print(train_data)
# Affichez les statistiques descriptives des données mises à l'échelle
print(train_data.describe())


# Pour la standardisation RobustScaler
test_data['Building Dimension'] = robust_scaler.fit_transform(test_data[['Building Dimension']])


In [ ]:
from sklearn.preprocessing import LabelEncoder
import pandas as pd

# Création d'une instance de LabelEncoder
label_encoder = LabelEncoder()

# Transformation des valeurs textuelles en valeurs numériques pour certaines colonnes
train_data["Building_Painted"] = label_encoder.fit_transform(train_data["Building_Painted"])
train_data["Building_Fenced"] = label_encoder.fit_transform(train_data["Building_Fenced"])
train_data["Garden"] = label_encoder.fit_transform(train_data["Garden"])
train_data["Settlement"] = label_encoder.fit_transform(train_data["Settlement"])
train_data["Insured_Period"] = label_encoder.fit_transform(train_data["Insured_Period"])
train_data["Building_Type"] = label_encoder.fit_transform(train_data["Building_Type"])





# Affichage du DataFrame final
print(train_data)


# Transformation des valeurs textuelles en valeurs numériques pour certaines colonnes
test_data["Building_Painted"] = label_encoder.fit_transform(test_data["Building_Painted"])
test_data["Building_Fenced"] = label_encoder.fit_transform(test_data["Building_Fenced"])
test_data["Garden"] = label_encoder.fit_transform(test_data["Garden"])
test_data["Settlement"] = label_encoder.fit_transform(test_data["Settlement"])
test_data["Insured_Period"] = label_encoder.fit_transform(test_data["Insured_Period"])
test_data["Building_Type"] = label_encoder.fit_transform(test_data["Building_Type"])


In [ ]:


from sklearn.preprocessing import KBinsDiscretizer

#'YearOfObservation'
"""
'YearOfObservation' a un coefficient d'asymétrie de 0.35, indiquant une légère asymétrie à droite.
Cependant, il n'y a pas d'outliers dans cette variable.
La méthode Equal-Width (Uniforme) est utilisée pour diviser la plage de la variable en intervalles de largeur égale.
Cette méthode est appropriée ici car la distribution est presque symétrique et sans outliers.
"""

#'Building Dimension'
"""
'Building Dimension' a un coefficient d'asymétrie de 1.13, indiquant une asymétrie à droite.
Il y a 396 outliers dans cette variable.
La méthode Equal-Frequency (Quantile) est utilisée pour diviser la plage de la variable de manière à ce que chaque intervalle contienne le même nombre d'observations.
Cette méthode est moins sensible aux valeurs aberrantes et convient donc mieux à cette colonne.
"""

# Création d'une instance de KBinsDiscretizer pour une discrétisation Equal-Width
discretizer_uniform = KBinsDiscretizer(n_bins=10, encode='ordinal', strategy='uniform')

# Discrétisation Equal-Width pour 'YearOfObservation'
train_data['YearOfObservation'] = discretizer_uniform.fit_transform(train_data[['YearOfObservation']])

# Création d'une instance de KBinsDiscretizer pour une discrétisation Equal-Frequency
discretizer_quantile = KBinsDiscretizer(n_bins=10, encode='ordinal', strategy='quantile')

# Discrétisation Equal-Frequency pour 'Building Dimension'
train_data['Building Dimension'] = discretizer_quantile.fit_transform(train_data[['Building Dimension']])


print(train_data)


In [ ]:
#Réduction
# Suppression de certaines colonnes qui ne seront pas utilisées
## Customer Id : Cet attribut semble être un identifiant unique pour chaque client.
# En général, nous n’utilisons pas les identifiants dans les modèles de machine learning
# car ils n’ont pas de pouvoir prédictif. Vous pouvez donc ignorer cet attribut.

train_data.drop(['Customer Id', 'Geo_Code', 'NumberOfWindows'], axis=1, inplace=True)



In [ ]:
train_data.corr()['Claim'].sort_values()


In [ ]:
test_data.drop(['Customer Id', 'Geo_Code', 'NumberOfWindows'], axis=1, inplace=True)


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier





# Séparer les caractéristiques et la cible
X_train = train_data.drop(['Claim'], axis=1)
y_train = train_data['Claim']
X_test = test_data.drop(['Claim'], axis=1)
y_test = test_data['Claim']




# Initialiser les modèles
models = [
    LogisticRegression(),
    DecisionTreeClassifier(),
    RandomForestClassifier(),
    SVC(),
    MLPClassifier()
]

# Entraîner et évaluer chaque modèle
for model in models:
    model.fit(X_train, y_train)
    # le modèle fait des prédictions sur X_test et compare ces prédictions à y_test
    #pour calculer un score de performance.
    score = model.score(X_test, y_test)
    print(f'Score for {model.__class__.__name__}: {score}')


from sklearn import metrics

# Initialiser les modèles
models = [
    LogisticRegression(),
    DecisionTreeClassifier(),
    RandomForestClassifier(),
    SVC(),
    MLPClassifier()
]

# Entraîner et évaluer chaque modèle
for model in models:
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    # Calcul des métriques de performance
    accuracy = metrics.accuracy_score(y_test, y_pred)
    precision = metrics.precision_score(y_test, y_pred, average='weighted')
    recall = metrics.recall_score(y_test, y_pred, average='weighted')
    f1_score = metrics.f1_score(y_test, y_pred, average='weighted')

    # Affichage des métriques de performance
    print(f'Metrics for {model.__class__.__name__}:')
    print(f'Accuracy: {accuracy}')
    print(f'Precision: {precision}')
    print(f'Recall: {recall}')
    print(f'F1 Score: {f1_score}\n')



In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc

# Initialiser les modèles
models = [
    LogisticRegression(),
    DecisionTreeClassifier(),
    RandomForestClassifier(),
    SVC(probability=True),
    MLPClassifier()
]

plt.figure(figsize=(10, 10))

# Entraîner et évaluer chaque modèle
for model in models:
    model.fit(X_train, y_train)
    # Prédire les probabilités de la classe positive
    y_score = model.predict_proba(X_test)[:, 1]
    # Calculer les points de la courbe ROC
    fpr, tpr, _ = roc_curve(y_test, y_score)
    # Calculer l'AUC
    roc_auc = auc(fpr, tpr)
    # Afficher la courbe ROC
    plt.plot(fpr, tpr, label=f'{model.__class__.__name__} (AUC = {roc_auc:.2f})')

# Afficher la ligne de hasard
plt.plot([0, 1], [0, 1], 'k--')

plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('Taux de faux positifs')
plt.ylabel('Taux de vrais positifs')
plt.title('Courbes ROC')
plt.legend(loc="lower right")
plt.show()




In [ ]:
# Commentaire sur LogisticRegression
"""
LogisticRegression a une AUC de 0.68, ce qui est la plus grande parmi tous les modèles.
Cela indique qu'il a la meilleure performance et a une plus grande probabilité de classer correctement un individu choisi au hasard.
"""

# Commentaire sur DecisionTreeClassifier
"""
DecisionTreeClassifier a une AUC de 0.49, ce qui est proche de celle d'un classificateur aléatoire (AUC = 0.5).
Il pourrait ne pas être très utile pour faire des prédictions précises.
"""

# Commentaire sur SVC
"""
SVC a une AUC de 0.50, ce qui est équivalent à celle d'un classificateur aléatoire.
Il pourrait ne pas être très utile pour faire des prédictions précises.
"""

# Commentaire sur RandomForrestClassifier
"""
RandomForrestClassifier a une AUC de 0.53, ce qui est légèrement meilleure que celle d'un classificateur aléatoire,
mais il est toujours loin derrière le modèle LogisticRegression.
"""

# Commentaire sur MLPClassifier
"""
MLPClassifier a la plus faible AUC parmi tous les modèles (AUC = 0.41), ce qui indique qu'il a la pire performance.
Il a une plus faible probabilité de classer correctement un individu choisi au hasard.
"""

# Résumé
"""
En résumé, le modèle LogisticRegression semble être le meilleur choix parmi ces modèles pour ce problème de classification spécifique,
selon les courbes ROC et les valeurs d'AUC. Cependant, il est important de noter que ces résultats sont spécifiques à l'ensemble de données
sur lequel les modèles ont été formés et testés. Les performances peuvent varier avec différents ensembles de données.
"""


In [ ]:
from sklearn.metrics import precision_recall_curve
import matplotlib.pyplot as plt

# Initialiser les modèles
models = [
    LogisticRegression(),
    DecisionTreeClassifier(),
    RandomForestClassifier(),
    SVC(probability=True),
    MLPClassifier()
]

# Pour chaque modèle
for model in models:
    # Entraîner le modèle
    model.fit(X_train, y_train)

    # Prédire les probabilités des classes
    y_scores = model.predict_proba(X_test)

    # Calculer les valeurs de précision et de rappel pour différents seuils
    precision, recall, thresholds = precision_recall_curve(y_test, y_scores[:, 1])

    # Afficher la courbe de précision-rappel
    plt.plot(recall, precision, label=model.__class__.__name__)

# Ajouter des détails au graphique
plt.xlabel('Rappel')
plt.ylabel('Précision')
plt.title('Courbe de précision-rappel')
plt.legend()
plt.show()


In [ ]:
from sklearn.metrics import average_precision_score
import matplotlib.pyplot as plt
from sklearn.metrics import PrecisionRecallDisplay

# Initialiser les modèles
models = [
    LogisticRegression(),
    DecisionTreeClassifier(),
    RandomForestClassifier(),
    SVC(probability=True),
    MLPClassifier()
]

# Pour chaque modèle
for model in models:
    # Entraîner le modèle
    model.fit(X_train, y_train)

    # Prédire les probabilités des classes
    y_scores = model.predict_proba(X_test)

    # Calculer le score de précision moyenne
    average_precision = average_precision_score(y_test, y_scores[:, 1])

    # Calculer les valeurs de précision et de rappel pour différents seuils
    disp = PrecisionRecallDisplay.from_estimator(model, X_test, y_test)

    # Afficher le score de précision moyenne
    print(f'Average precision-recall score for {model.__class__.__name__}: {average_precision}')

# Afficher la courbe de précision-rappel
plt.show()


In [ ]:
# Commentaire sur LogisticRegression
"""
LogisticRegression a un score moyen de précision-rappel de 0.383, ce qui est le plus élevé parmi tous les modèles.
Cela indique qu'il a la meilleure performance en termes de compromis entre la précision et le rappel.
"""

# Commentaire sur DecisionTreeClassifier
"""
DecisionTreeClassifier a un score moyen de précision-rappel de 0.234, ce qui est relativement faible.
Cela indique qu'il pourrait ne pas être très efficace pour équilibrer la précision et le rappel.
"""

# Commentaire sur RandomForestClassifier
"""
RandomForestClassifier a un score moyen de précision-rappel de 0.241, ce qui est légèrement supérieur à celui du DecisionTreeClassifier,
mais il est toujours assez faible.
"""

# Commentaire sur SVC
"""
SVC a un score moyen de précision-rappel de 0.236, ce qui est similaire à celui du DecisionTreeClassifier et du RandomForestClassifier.
"""

# Commentaire sur MLPClassifier
"""
MLPClassifier a le score moyen de précision-rappel le plus faible parmi tous les modèles (0.222),
ce qui indique qu'il a la pire performance en termes de compromis entre la précision et le rappel.
"""

# Résumé
"""
En résumé, le modèle LogisticRegression semble être le meilleur choix parmi ces modèles pour ce problème de classification spécifique,
selon les scores moyens de précision-rappel. Cependant, il est important de noter que ces résultats sont spécifiques à l'ensemble de données
sur lequel les modèles ont été formés et testés. Les performances peuvent varier avec différents ensembles de données.
"""
